In [1]:
import pandas as pd
import requests
import unidecode
import numpy as np
import getpass
from datetime import datetime, timedelta
from tqdm import tqdm

import warnings   
warnings.filterwarnings("ignore")

In [2]:
def run_query(query,query_name = 'Sorgu'):
    import cx_Oracle
    from datetime import datetime 
    
    # ORACLE CONNECTION INFO
    CONN_INFO = {'host': YOUR_HOST,
                 'port': YOUR_PORT,
                 'user': YOUR_USER,
                 'psw': YOUR_PASSWORD,
                 'service': YOUR_SERVICE}

    #print(f'\n{CONN_INFO["service"]} servisine baglaniliyor...')
    CONN_STR = '{user}/{psw}@{host}:{port}/{service}'.format(**CONN_INFO)

    con = cx_Oracle.connect(CONN_STR)
    #print(f'{CONN_INFO["service"]} servisine bağlanıldı.')

    begin = datetime.now()
    print(f'\n{query_name} koşuyor.. Başlama saati {begin.hour}:{begin.minute}:{begin.second}')

    data = pd.read_sql(query, con)

    end = datetime.now()
    delta = end - begin
    print(f'\t{query_name} {round(delta.total_seconds(),1)} saniyede koştu.\n')

    #print(data.tail(5)) #Son 5 Satırı getir.

    con.close()

    return data

In [3]:
username = getpass.getuser()
username

'mert.eygi'

In [4]:
ACCESS_TOKEN = YOUR_TIKTOK_ACCESS_TOKEN
ADVERTISER_ID = YOUR_TIKTOK_ADVERTISER_ID

In [5]:
headers = {
    'Access-Token': ACCESS_TOKEN,
}

# 1. Get PAGE_IDs

https://ads.tiktok.com/marketing_api/docs?id=1735735588640770

In [6]:
## GET PAGE_IDs
params = {
    'advertiser_id': ADVERTISER_ID,
    #'fields': '["FIELDS"]',
    #'filtering': '{"status": "STATUS", "adgroup_ids": ["ADGROUP_IDS"], "adgroup_name": "ADGROUP_NAME", "campaign_ids": ["CAMPAIGN_IDS"], "billing_events": ["BILLING_EVENTS"], "primary_status": "PRIMARY_STATUS", "objective_type": "OBJECTIVE_TYPE"}',
    #'page': '{{PAGE}}',
    #'page_size': '{{PAGE_SIZE}}',
    'page_size': '1000'
}

response = requests.get('https://business-api.tiktok.com/open_api/v1.3/ad/get/', params=params, headers=headers)
print(f'response :\n{response}')

print(f"Page Info:\n{response.json()['data']['page_info']}")

#Desired columns to get all page_ids
cols = ['advertiser_id','campaign_id','campaign_name','adgroup_id', 'adgroup_name','ad_id','ad_name','page_id','create_time']

df_get_ads = pd.DataFrame(response.json()['data']['list'])[cols]
#print(df_get_ads.shape)
#df_get_ads.head()

response :
<Response [200]>
Page Info:
{'page': 1, 'page_size': 1000, 'total_page': 1, 'total_number': 180}


# 2. Get TASK_IDs

In [7]:
## GET TASK_IDs 

params_create_task = {
    'advertiser_id': ADVERTISER_ID,
    'page_id': '0'
}

params_dl_lead = {
    'advertiser_id': ADVERTISER_ID,
    'task_id': '0'
}


task_ids = []

count=0
for p in tqdm(df_get_ads['page_id'].unique()):
    #print(f'---------------task_id: {p},count:{count}------------------')
    params_create_task['page_id'] = p
    
    rsp_create_task = requests.post('https://ads.tiktok.com/open_api/v1.3/page/lead/task/', 
                                    params=params_create_task, 
                                    headers=headers).json()

    #print(rsp_create_task)

    TASK_ID = rsp_create_task['data']['task_id']
    task_ids.append(TASK_ID)
    count += 1

100%|██████████████████████████████████████████████████████████████████████████████████| 82/82 [00:36<00:00,  2.26it/s]


# 3. Collect All Leads

In [8]:
# COLLECT ALL LEADs
count=0
cols_all_leads = []
df_tiktok_leads = pd.DataFrame()
for t in tqdm(task_ids):
    #print(f'---------------task_id: {t},count:{count}------------------')
    params_dl_lead['task_id'] = t
    
    lead_res = requests.get('https://business-api.tiktok.com/open_api/v1.3/page/lead/task/download/', 
                        params=params_dl_lead, 
                        headers=headers)
    #print(f'response:\t{lead_res}')
    
    lead_res_list = lead_res.text.split('\n')
    
    if lead_res_list != ['']:
        #print(lead_res_list[2].find('"'))
        cols_all_leads.append(lead_res_list[0].split(','))

        
        lead_res_list2 = []
        for lead_row in lead_res_list:
            # if the string not contains (").
            if lead_row.find('"')==-1: 
                lead_row_splitted = lead_row.split(',')
            # if the string contains (").    
            else: 
                lead_split = lead_row.split('"')
                if len(lead_split)==3:
                    lead_row_splitted = lead_split[0][:-1].split(',') + [lead_split[1]] + lead_split[2][1:].split(',')
                else:
                    print(f'Something Different, len(lead_split): {len(lead_split)}')
                    lead_row_splitted = ['']
            lead_res_list2.append(lead_row_splitted)
        
        
        temp_lead_df = pd.DataFrame(lead_res_list2)
        temp_lead_df.columns = temp_lead_df.iloc[0]
        #display(temp_lead_df)
        temp_lead_df.dropna(thresh=10,inplace=True)
        temp_lead_df.drop(index=0,axis=0,inplace=True)
        temp_lead_df.reset_index(drop=True,inplace=True)

        #print(f'temp_lead_df.shape: {temp_lead_df.shape}')
        df_tiktok_leads = pd.concat([df_tiktok_leads,temp_lead_df],axis=0)
    
    count += 1


print(df_tiktok_leads.shape)
df_tiktok_leads.reset_index(inplace=True,drop=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 82/82 [01:07<00:00,  1.21it/s]

(4192, 15)


# 4. Edit The TikTok Leads

In [9]:
df_tiktok_orj = df_tiktok_leads.copy()

In [10]:
df_tiktok_leads = df_tiktok_orj.copy()

In [1]:
#print(df_tiktok_leads.shape)
#df_tiktok_leads.head()

In [12]:
df_tiktok_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4192 entries, 0 to 4191
Data columns (total 15 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   lead_id                                           4192 non-null   object
 1   created_time                                      4192 non-null   object
 2   ad_id                                             4192 non-null   object
 3   ad_name                                           4192 non-null   object
 4   adgroup_id                                        4192 non-null   object
 5   adgroup_name                                      4192 non-null   object
 6   campaign_id                                       4192 non-null   object
 7   campaign_name                                     4192 non-null   object
 8   form_id                                           4192 non-null   object
 9   form_name                     

In [13]:
df_tiktok_leads.columns

Index(['lead_id', 'created_time', 'ad_id', 'ad_name', 'adgroup_id',
       'adgroup_name', 'campaign_id', 'campaign_name', 'form_id', 'form_name',
       'Ad', 'E-posta', 'Telefon numarası',
       'Hangi Kulübümüzle İlgili Bilgi Almak İstersiniz?',
       'Hangi Kulübümüzle İlgili Bilgi Almak İstersin?'],
      dtype='object', name=0)

In [14]:
df_tiktok_leads['CLUB'] = 'GENEL'

for i in df_tiktok_leads.columns:
    i2 = unidecode.unidecode(i.lower()) # convert non-english chars to eng chars
    if ('kulub' in i2) or ('club' in i2) or ('kulup' in i2):
        df_tiktok_leads.loc[~df_tiktok_leads[i].isna(),'CLUB'] = df_tiktok_leads.loc[~df_tiktok_leads[i].isna(),i].str.lower().str.replace(' ','_')

In [15]:
df_tiktok_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4192 entries, 0 to 4191
Data columns (total 16 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   lead_id                                           4192 non-null   object
 1   created_time                                      4192 non-null   object
 2   ad_id                                             4192 non-null   object
 3   ad_name                                           4192 non-null   object
 4   adgroup_id                                        4192 non-null   object
 5   adgroup_name                                      4192 non-null   object
 6   campaign_id                                       4192 non-null   object
 7   campaign_name                                     4192 non-null   object
 8   form_id                                           4192 non-null   object
 9   form_name                     

In [16]:
df_tiktok_leads['CLUB']

0             torium_(esenyurt)
1                vialand_(eyüp)
2               cevahir_(şişli)
3          nevçarşı_üsküdar_avm
4             neomarin_(pendik)
                 ...           
4187           macfit_ege_perla
4188       macfit_point_bornova
4189       macfit_point_bornova
4190    macfit_gaziemir_optimum
4191            macfit_kidsmall
Name: CLUB, Length: 4192, dtype: object

In [17]:
df_tiktok_leads.rename(columns={'Ad': 'NAME', 'E-posta': 'EMAIL'},inplace=True)

# Update column names with its uppercase
df_tiktok_leads.columns = [x.upper() for x in df_tiktok_leads.columns]

df_tiktok_leads['LEAD_DAY'] = df_tiktok_leads['CREATED_TIME'].apply(lambda x: x.split(' ')[0])
df_tiktok_leads['PHONE'] = df_tiktok_leads['TELEFON NUMARASI'].apply(lambda x: x.replace(' ','').replace('-',''))



df_tiktok_leads = df_tiktok_leads.astype({'LEAD_ID': 'int64','AD_ID': 'int64','ADGROUP_ID': 'int64',
                                          'CAMPAIGN_ID' : 'int64','FORM_ID': 'int64','LEAD_DAY' : 'datetime64[ns]'
                                         })




df_tiktok_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4192 entries, 0 to 4191
Data columns (total 18 columns):
 #   Column                                            Non-Null Count  Dtype         
---  ------                                            --------------  -----         
 0   LEAD_ID                                           4192 non-null   int64         
 1   CREATED_TIME                                      4192 non-null   object        
 2   AD_ID                                             4192 non-null   int64         
 3   AD_NAME                                           4192 non-null   object        
 4   ADGROUP_ID                                        4192 non-null   int64         
 5   ADGROUP_NAME                                      4192 non-null   object        
 6   CAMPAIGN_ID                                       4192 non-null   int64         
 7   CAMPAIGN_NAME                                     4192 non-null   object        
 8   FORM_ID                     

In [18]:
## Bu kısım LEAD_ADS ile birleştirilirken çıkartılabilir
#########
current_location = 'C:\\Users\\'+ username +'\\Documents\\GitHub\\macdigital\\Leads\\'
club_lookup = pd.read_excel(current_location + 'CLUB_NAME_LOOKUP.xlsx')


club_lookup_yeni = club_lookup.query("YENI_ESKI == 'YENI' ")
club_lookup_yeni_1 = club_lookup_yeni.copy()
club_lookup_yeni_2 = club_lookup_yeni.copy()
club_lookup_yeni_1['DIGITAL_MARKETING_NAME'] = club_lookup_yeni['DIGITAL_MARKETING_NAME'].str.lower().str.replace(' ',"_")
club_lookup_yeni_2['DIGITAL_MARKETING_NAME'] = club_lookup_yeni['DB_CLUB_TYPE']+ '_' + club_lookup_yeni['DIGITAL_MARKETING_NAME'].str.lower().str.replace(' ',"_")

club_lookup_final = pd.concat([club_lookup,
          club_lookup_yeni_1,
          club_lookup_yeni_2
          ],axis=0
         )

club_lookup_final.drop_duplicates(subset=['DIGITAL_MARKETING_NAME'],inplace=True)
#########

club_lookup_tiktok = club_lookup_final.copy()
club_lookup_tiktok['DIGITAL_MARKETING_NAME'] = club_lookup_tiktok['DIGITAL_MARKETING_NAME'].str.lower().str.replace(' ','_')
club_lookup_tiktok.drop_duplicates(subset=['DIGITAL_MARKETING_NAME'],inplace=True)
print(club_lookup_tiktok.shape)
club_lookup_tiktok.sample(5)

(295, 7)


,DIGITAL_MARKETING_NAME,DB_CLUB_ID,DB_SHORT_NAME,DB_CLUB_TYPE,YENI_ESKI,DB_CITY_NAME,DB_NAME
42,fulya,14,021 Ist Fulya,macfit,YENI,İstanbul,Fulya
160,mac_zorlu,92,089 Ist Zorlu Center,mac,ESKI,NaN,NaN
197,macfit_nişantaşı_lotus,109,104 Ist Nisantasi Lotus,macfit,ESKI,NaN,NaN
82,macfit_podium_(yenimahalle),35,034 Ank Podium,macfit,YENI,Ankara,Podium
180,macfit_kıbrıs_sehitleri,95,092 Izm Kibris Sehitleri,macfit,ESKI,NaN,NaN


In [19]:
club_lookup

,DIGITAL_MARKETING_NAME,DB_CLUB_ID,DB_SHORT_NAME,DB_CLUB_TYPE,YENI_ESKI,DB_CITY_NAME,DB_NAME
0,01 Burda (Adana),101,097 Ada 01 Burda,macfit,YENI,Adana,01 Burda
1,365 AVM (Maltepe),106,103 Ank 365 AVM,macfit,YENI,Ankara,365 AVM
2,41 Burda (Kocaeli),37,037 Koc 41 Burda,macfit,YENI,Kocaeli,41 Burda
3,42 Maslak,58,058 Ist 42 Maslak,macfit,YENI,İstanbul,42 Maslak
4,A Plus (Ataköy),17,006 Ist A Plus,mac,YENI,İstanbul,A Plus
...,...,...,...,...,...,...,...
240,macfit_nilüfer_carrefour_bursa,10,017 Bur Nilufer Carrefour,macfit,ESKI,NaN,NaN
241,macfit_podyumpark_bursa,15,022 Bur Podyumpark,macfit,ESKI,NaN,NaN
242,macfit_örs_plaza_mersin,82,080 Mer Ors Plaza,macfit,ESKI,NaN,NaN
243,nilüfer_carrefour_(bursa),10,017 Bur Nilufer Carrefour,macfit,ESKI,NaN,NaN


In [20]:
club_lookup_tiktok

,DIGITAL_MARKETING_NAME,DB_CLUB_ID,DB_SHORT_NAME,DB_CLUB_TYPE,YENI_ESKI,DB_CITY_NAME,DB_NAME
0,01_burda_(adana),101,097 Ada 01 Burda,macfit,YENI,Adana,01 Burda
1,365_avm_(maltepe),106,103 Ank 365 AVM,macfit,YENI,Ankara,365 AVM
2,41_burda_(kocaeli),37,037 Koc 41 Burda,macfit,YENI,Kocaeli,41 Burda
3,42_maslak,58,058 Ist 42 Maslak,macfit,YENI,İstanbul,42 Maslak
4,a_plus_(ataköy),17,006 Ist A Plus,mac,YENI,İstanbul,A Plus
...,...,...,...,...,...,...,...
92,macfit_vadistanbul_(sarıyer),90,094 Ist Umraniye Cagla Plaza,macfit,YENI,İstanbul,Vadistanbul
93,macfit_vega_(sultangazi),105,102 Ist Vega,macfit,YENI,İstanbul,Vega
94,macfit_vialand_(eyüp),8,015 Ist Vialand,macfit,YENI,İstanbul,Vialand
95,macfit_watergarden_(ataşehir),47,043 Ist Watergarden,macfit,YENI,İstanbul,Watergarden


In [21]:
df_tiktok_leads.shape

(4192, 18)

In [22]:
df_tiktok_leads = pd.merge(df_tiktok_leads,
                           club_lookup_tiktok[['DIGITAL_MARKETING_NAME', 'DB_CLUB_ID', 'DB_SHORT_NAME']],
                           left_on = 'CLUB', right_on = 'DIGITAL_MARKETING_NAME',how='left'
                          )

df_tiktok_leads.rename(columns={'DB_CLUB_ID': 'LEAD_CLUB_ID', 'DB_SHORT_NAME': 'LEAD_SHORT_NAME' },inplace=True)

df_tiktok_leads.shape

(4192, 21)

In [23]:
df_tiktok_leads.isna().sum()

LEAD_ID                                                0
CREATED_TIME                                           0
AD_ID                                                  0
AD_NAME                                                0
ADGROUP_ID                                             0
ADGROUP_NAME                                           0
CAMPAIGN_ID                                            0
CAMPAIGN_NAME                                          0
FORM_ID                                                0
FORM_NAME                                              0
NAME                                                   0
EMAIL                                                  0
TELEFON NUMARASI                                       0
HANGI KULÜBÜMÜZLE İLGILI BILGI ALMAK İSTERSINIZ?    3979
HANGI KULÜBÜMÜZLE İLGILI BILGI ALMAK İSTERSIN?       410
CLUB                                                   0
LEAD_DAY                                               0
PHONE                          

In [24]:
df_tiktok_leads.columns

Index(['LEAD_ID', 'CREATED_TIME', 'AD_ID', 'AD_NAME', 'ADGROUP_ID',
       'ADGROUP_NAME', 'CAMPAIGN_ID', 'CAMPAIGN_NAME', 'FORM_ID', 'FORM_NAME',
       'NAME', 'EMAIL', 'TELEFON NUMARASI',
       'HANGI KULÜBÜMÜZLE İLGILI BILGI ALMAK İSTERSINIZ?',
       'HANGI KULÜBÜMÜZLE İLGILI BILGI ALMAK İSTERSIN?', 'CLUB', 'LEAD_DAY',
       'PHONE', 'DIGITAL_MARKETING_NAME', 'LEAD_CLUB_ID', 'LEAD_SHORT_NAME'],
      dtype='object')

In [25]:
df_tiktok_leads.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4192 entries, 0 to 4191
Data columns (total 21 columns):
 #   Column                                            Non-Null Count  Dtype         
---  ------                                            --------------  -----         
 0   LEAD_ID                                           4192 non-null   int64         
 1   CREATED_TIME                                      4192 non-null   object        
 2   AD_ID                                             4192 non-null   int64         
 3   AD_NAME                                           4192 non-null   object        
 4   ADGROUP_ID                                        4192 non-null   int64         
 5   ADGROUP_NAME                                      4192 non-null   object        
 6   CAMPAIGN_ID                                       4192 non-null   int64         
 7   CAMPAIGN_NAME                                     4192 non-null   object        
 8   FORM_ID                     

In [26]:
#df_tiktok_leads.to_excel('tiktok_leads_6.xlsx')

In [27]:
def insert_dwh_tiktok(records):
    
    ### CREATE ORACLE SQL INSERT FUNCTIONS ###
    import cx_Oracle
    from datetime import datetime

    # ORACLE CONNECTION INFO
    CONN_INFO = {'host': YOUR_HOST,
                 'port': YOUR_PORT,
                 'user': YOUR_USER,
                 'psw': YOUR_PASSWORD,
                 'service': YOUR_SERVICE}

    CONN_STR = '{user}/{psw}@{host}:{port}/{service}'.format(**CONN_INFO)

    """
    insert multiple records
    :param records: a list of records
    :return:
    """
    # construct an insert statement that add a new row to the DWH.DIGITAL_LEADS_TIKTOK table
    sql = ('insert into DWH.DIGITAL_LEADS_TIKTOK '
        'values(:LEAD_ID,:AD_ID,:AD_NAME,:ADGROUP_ID,:ADGROUP_NAME,:CAMPAIGN_ID,:CAMPAIGN_NAME,:FORM_ID,:FORM_NAME,'
        ':LEAD_DAY,:NAME,:PHONE,:EMAIL,:CLUB,:LEAD_CLUB_ID,:LEAD_SHORT_NAME,:ETL_DATE)')
    #print(sql)

    try:
        # establish a new connection
        with cx_Oracle.connect(CONN_STR) as connection:
            # create a cursor
            with connection.cursor() as cursor:
                print('\nAll Facebook Leads are inserting to "DWH.DIGITAL_LEADS_TIKTOK" table ')
                time = datetime.now()
                print(f'Starting Time: {time.year}-{time.month}-{time.day},{time.hour}:{time.minute}:{time.second}')

                #### cursor.executemany operation #### 
                cursor.executemany(sql, records)                

                ### COMMITING WORK ###
                connection.commit()
                print(f'\tTABLE INSERT IS COMPLETED: {len(records)} rows are inserted to "DWH.DIGITAL_LEADS_TIKTOK" table')
                time = datetime.now()
                print(f'Ending Time: {time.year}-{time.month}-{time.day},{time.hour}:{time.minute}:{time.second}')
    except cx_Oracle.Error as error:
        print('Error occurred:')
        print(error)

######################################
##### INSERT LEADS TO DWH ############

In [29]:
#desired column names
db_cols = ['LEAD_ID','AD_ID','AD_NAME','ADGROUP_ID','ADGROUP_NAME',
           'CAMPAIGN_ID','CAMPAIGN_NAME','FORM_ID','FORM_NAME','LEAD_DAY',
           'NAME','PHONE','EMAIL','CLUB','LEAD_CLUB_ID','LEAD_SHORT_NAME']

df_tiktok_leads_prev = run_query('select * from DWH.DIGITAL_LEADS_TIKTOK','Table Count')

df_tiktok_leads_old = df_tiktok_leads_prev[db_cols].copy()
df_tiktok_leads_new = df_tiktok_leads[db_cols].copy()
df_tiktok_leads_new.reset_index(drop=True,inplace=True)

print(f'df_tiktok_leads_old shape : {df_tiktok_leads_old.shape}')
print(f'df_tiktok_leads_new shape : {df_tiktok_leads_new.shape}')     

df_insert = df_tiktok_leads_old.merge(df_tiktok_leads_new, on=db_cols, how='outer', indicator=True)

df_insert = df_insert[df_insert['_merge']=='right_only']
print(f'df_insert shape : {df_insert.shape}')
df_insert.reset_index(drop=True,inplace=True)


#If there are new rows, then insert them
if len(df_insert) > 0:

    #insertList = list(insert_df.to_records(index=False))
    insertList=[]
    nonInsertList = []
    etl_date = datetime.now()

    for i in range(len(df_insert)):
        ## if that row doesn't have any "nan" value
        if df_insert.loc[i].isna().sum()==0:
            ts_lead_day = df_insert.loc[i,'LEAD_DAY']

            lead_id = int(df_insert.loc[i,'LEAD_ID'])
            ad_id = int(df_insert.loc[i,'AD_ID'])
            ad_name = df_insert.loc[i,'AD_NAME']
            adgroup_id = int(df_insert.loc[i,'ADGROUP_ID'])
            adgroup_name = df_insert.loc[i,'ADGROUP_NAME']
            campaign_id = int(df_insert.loc[i,'CAMPAIGN_ID'])
            campaign_name = df_insert.loc[i,'CAMPAIGN_NAME']
            form_id = int(df_insert.loc[i,'FORM_ID'])
            form_name = df_insert.loc[i,'FORM_NAME']
            lead_day = datetime(ts_lead_day.year,ts_lead_day.month,ts_lead_day.day)
            name = df_insert.loc[i,'NAME']
            phone = df_insert.loc[i,'PHONE']
            email = df_insert.loc[i,'EMAIL']
            club = df_insert.loc[i,'CLUB']
            lead_club_id = int(df_insert.loc[i,'LEAD_CLUB_ID'])
            lead_short_name = df_insert.loc[i,'LEAD_SHORT_NAME']


            insertList.append((lead_id, ad_id, ad_name, adgroup_id, adgroup_name, 
                               campaign_id, campaign_name, form_id, form_name, 
                               lead_day, name, phone, email, club, 
                               lead_club_id, lead_short_name, etl_date))
        ## if that row have a "nan" value
        else:
            nonInsertList.append(list(df_insert.loc[i].values))

    insert_dwh_tiktok(insertList)
    if len(nonInsertList)>0:
        df_nonInsert = pd.DataFrame(nonInsertList,columns=df_insert.columns)
        print(f"\n !!! {len(nonInsertList)} Tiktok leads didn't be inserted because they have 'nan' values, check: 'df_nonInsert'")
else:
    print("\n***** TIKTOK LEADS DIDN'T BE INSERTED BECASUE THERE IS NO NEW RECORDS *****")


Table Count koşuyor.. Başlama saati 9:50:57
	Table Count 3.8 saniyede koştu.

df_tiktok_leads_old shape : (4317, 16)
df_tiktok_leads_new shape : (4192, 16)
df_insert shape : (0, 17)

***** TIKTOK LEADS DIDN'T BE INSERTED BECASUE THERE IS NO NEW RECORDS *****
